In [1]:
import openmatrix as omx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas

In [2]:
# Have matplotlib open "plots"  in a separate window
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
trip_tables_file = r'C:/Users/ben_k/work_stuff/tdm/datastore/sample_data/AfterSC_Final_AM_Tables.omx'

In [4]:
trip_tables_omx = omx.open_file(trip_tables_file, 'r')

In [5]:
trip_tables_omx.list_matrices()

['Bike',
 'DAT_Boat',
 'DAT_CR',
 'DAT_LB',
 'DAT_RT',
 'DET_Boat',
 'DET_CR',
 'DET_LB',
 'DET_RT',
 'HOV',
 'HOV2p',
 'HOV3p',
 'HOV_Person_Trips',
 'Heavy_Truck',
 'Heavy_Truck_HazMat',
 'Light_Truck',
 'Medium_Truck',
 'Medium_Truck_HazMat',
 'SOV',
 'WAT',
 'Walk']

In [6]:
trip_tables_omx.shape()

(5839, 5839)

In [7]:
trip_tables_omx.list_mappings()

['CTPS', 'ID', 'MAPC', 'extZ', 'intZ']

In [8]:
taz_to_omxid = trip_tables_omx.mapping('ID')

In [9]:
skim_file = r'C:/Users/ben_k/work_stuff/tdm/datastore/sample_data/AM_SOV_Skim.omx'

In [10]:
skim_omx = omx.open_file(skim_file, 'r')

In [11]:
skim_omx.list_matrices()

['Auto_Toll (Skim)',
 'CongTime',
 'CongTime_wTerminalTimes',
 'Drive_Cost',
 'LT_Toll (Skim)',
 'Length (Skim)',
 'TerminalTimes',
 'Total_Cost']

In [12]:
skim_omx.shape()

(5839, 5839)

In [13]:
skim_omx.list_mappings()

['Destination', 'Origin', 'extZ', 'inCTPS', 'intZ']

In [14]:
# (1) Get A.M. trip length distribution by mode [SOV, HOV, Bike, Walk]
trip_length_by_mode = {}
modes = [ 'SOV', 'HOV', 'Bike', 'Walk']
for mode in modes:
    tmp1 = np.multiply(skim_omx['Length (Skim)'], trip_tables_omx[mode])
    tmp2 = np.sum(tmp1)
    # s = mode + ' trip length = ' + str(tmp1)
    # print(s)
    trip_length_by_mode[mode]= tmp2

In [16]:
# (2) Plot the data as a bar chart
names = trip_length_by_mode.keys()
values = trip_length_by_mode.values()
scaled_values = []
for value in values:
    scaled_values.append(value / 10e6)
plt.title('Trip Length Distribution by Mode')
plt.xlabel('Transportation Mode')
plt.ylabel('Trip Length in 10^6 miles')
plt.bar(names, scaled_values)
# The following line is not needed in an IPython Notebook environment
# plt.show()

<BarContainer object of 4 artists>

In [17]:
# (3) Calculate link VMT by functional class of road
flow_fn = r'C:/Users/ben_k/work_stuff/tdm/datastore/sample_data/AM_MMA_LinkFlow.csv'
links_fn = r'C:/Users/ben_k/work_stuff/tdm/datastore/sample_data/statewide_links_pruned.csv'
df_flow = pd.read_csv(flow_fn, usecols=['ID1', 'Tot_Flow'], dtype={'ID1':np.int32, 'Tot_Flow':np.float64})
df_links = pd.read_csv(links_fn, usecols=['ID', 'SCEN_00_FU'])

In [18]:
joined_data = df_links.set_index('ID').join(df_flow.set_index('ID1'))

In [19]:
total_flow_by_fc = joined_data.groupby('SCEN_00_FU')['Tot_Flow'].sum()

In [20]:
fc_names = df_links.groupby(['SCEN_00_FU']).groups.keys()

In [21]:
pruned_fc_names = []
pruned_total_flow_by_fc = [] 

In [22]:
# Here ASERT len(fc_names) == len(total_flow_by_fc)
if len(fc_names) != len(total_flow_by_fc):
    print("Something is wrong:")
    s = '    Length of fc_names = ' + str(len(fc_names))
    print(s)
    s = '    Length of total_flow_by_fc' + str(len(total_flow_by_fc))
    print(s)

In [23]:
# Do the actual pruning
for (x,y) in zip(fc_names, total_flow_by_fc):
    if x < 10:
        pruned_fc_names.append(x)
        pruned_total_flow_by_fc.append(y)

In [24]:
# (4) Generate the plot of link VMT by functional class        
scaled_values = []
for value in pruned_total_flow_by_fc:
    scaled_values.append(value / 10e7)

In [25]:
plt.title('Link VMT by Functional Class')
plt.xlabel('Functional Class')
plt.ylabel('Link VMT in 10^7 Miles')
plt.bar(pruned_fc_names, scaled_values)
# plt.show()

<BarContainer object of 5 artists>

In [26]:
# (5.1) Generate a map of the TAZes, symbolized by state
base = r'C:/Users/ben_k/work_stuff/tdm/datastore/reference_data/'

taz_shpfile = 'candidate_CTPS_TAZ_STATE_2019.shp'
fn = base + taz_shpfile
gdf = geopandas.read_file(fn)
gdf.set_index("id")
gdf.plot("state", figsize=(10.0,8.0), legend=True)

<AxesSubplot:>

In [27]:
# (5.2) Generate a map of ALL the links, symbolized by functional class
links_shpfile = 'Statewide_Links_2018_BK_EPSG26986.shp'
fn2 = base + links_shpfile
gdf2 = geopandas.read_file(fn2)
gdf2.set_index("ID")
gdf2.plot("SCEN_00_FU", figsize=(10.0,8.0), legend=True)

<AxesSubplot:>

In [28]:
# (5.3) Generate a map of all links with a "reasonable" functional class
real_roads = gdf2[gdf2["SCEN_00_FU"] < 10]
real_roads.plot(column="SCEN_00_FU", categorical=True, legend=True, figsize=(10.0,8.0))

<AxesSubplot:>

In [29]:
# (5.4) Join links data for "real roads"to flow data on 'ID'/'ID1' fields,
#       and generate a map of relative flow by link.
real_roads_joined = real_roads.set_index('ID').join(df_flow.set_index('ID1'))

In [30]:
real_roads_joined.head()

,DIR,LENGTH,STATE,ANODE,BNODE,TAZ_ID,STREETNAME,ROUTENUMBE,SCEN_00,SCEN_00_FU,geometry,Tot_Flow
ID,,,,,,,,,,,,
200648,0,0.303488,RI,160862.0,160893.0,208448,WALLUM LAKE RD,None,1,5,"LINESTRING (178534.494 860032.322, 178535.226 ...",817.206595
200637,0,0.051294,RI,160893.0,160894.0,208448,WALLUM LAKE RD,None,1,5,"LINESTRING (178594.130 860516.853, 178595.411 ...",817.206595
200654,0,0.121481,RI,160871.0,160834.0,208448,WALLUM LAKE RD,None,1,5,"LINESTRING (178447.503 860933.368, 178444.955 ...",817.206595
200617,0,0.107739,RI,160870.0,160871.0,208448,WALLUM LAKE RD,None,1,5,"LINESTRING (178531.717 860781.945, 178497.785 ...",817.206595
200671,0,0.120546,RI,160894.0,160870.0,208448,WALLUM LAKE RD,None,1,5,"LINESTRING (178592.473 860599.050, 178591.830 ...",817.206595


In [31]:
real_roads_joined.plot(column='Tot_Flow', legend=True, figsize=(10.0,8.0))

<AxesSubplot:>

In [32]:
real_roads_joined.describe()

,DIR,LENGTH,ANODE,BNODE,TAZ_ID,SCEN_00,SCEN_00_FU,Tot_Flow
count,152728.000000,152728.000000,152728.000000,152728.000000,152728.000000,152728.000000,152728.000000,152610.000000
mean,0.144414,0.109764,109229.855115,109204.677603,40573.511386,0.999293,4.670473,1878.824922
std,0.366519,0.167405,64579.248020,64567.152564,79521.541041,0.026583,1.287324,2022.827416
min,-1.000000,0.000118,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.033120,53105.750000,53119.750000,1588.000000,1.000000,3.000000,559.639001
50%,0.000000,0.058086,105940.500000,105884.500000,3114.000000,1.000000,5.000000,1412.003636
75%,0.000000,0.118499,158557.250000,158519.250000,5536.000000,1.000000,6.000000,2572.433132
max,1.000000,4.456859,231106.000000,231106.000000,208992.000000,1.000000,6.000000,28912.072058


In [33]:
def classify_flow(flow):
	retval = 0
	if flow == None:
		retval = 0
	elif flow < 100.0:
		retval = 1
	elif flow < 500.0:
		retval = 2
	elif flow < 1000.0:
		retval = 3
	elif flow < 5000.0:
		retval = 4
	elif flow < 10000.0:
		retval = 5
	else:
		retval = 6
	# end_if
	return retval
# end_def 

In [34]:
rv = real_roads_joined.assign(flow_class=0)

In [35]:
rv['flow_class'] = rv.apply(lambda row:classify_flow(row['Tot_Flow']), axis=1)

In [36]:
# (5.5) Generate a map of 6-way classifcation of flow by link
rv.plot(column="flow_class", categorical=True, legend=True, figsize=(10.0,8.0))

<AxesSubplot:>

In [37]:
# (5.6) Generate a map of the geometrically "simplified" TAZes, symbolized by state
simp_taz_shpfile = 'candidate_CTPS_TAZ_STATE_2019_simp_10m.shp'
fn3 = base + simp_taz_shpfile
gdf3 = geopandas.read_file(fn3)
gdf3.set_index("id")
gdf3.plot("state", figsize=(10.0,8.0), legend=True)

<AxesSubplot:>

In [38]:
# (5.7) Generate a map of ALL the geometrically "simplified" links symbolized by functional class
simp_links_shpfile = 'Statewide_Links_2018_BK_EPSG26986_simp_10m.shp'
fn4 = base + simp_links_shpfile
gdf4 = geopandas.read_file(fn4)
gdf4.plot("SCEN_00_FU", figsize=(10.0,8.0), legend=True)

<AxesSubplot:>

In [39]:
# (5.8) Generate a map of all the "simplified" links with a "reasonable" functional class
real_roads_simp = gdf4[gdf4["SCEN_00_FU"] < 10]
real_roads_simp.plot(column="SCEN_00_FU", categorical=True, legend=True, figsize=(10.0,8.0))

<AxesSubplot:>

In [40]:
# (5.9) Join links data for simplified "real roads" to flow data on 'ID'/'ID1' fields,
#		and generate a map of 6-way classifcation of flow by link
real_roads_simp_joined = real_roads_simp.set_index('ID').join(df_flow.set_index('ID1'))
real_roads_simp_joined2 = real_roads_simp_joined.assign(flow_class=0)
real_roads_simp_joined2['flow_class'] = real_roads_simp_joined2.apply(lambda row: classify_flow(row['Tot_Flow']), axis=1)
real_roads_simp_joined2.plot(column="flow_class", categorical=True, legend=True, figsize=(10.0,8.0))

<AxesSubplot:>

In [41]:
# (5.10) Same as 5.9, but using a somewhat more meaningful symbolization.
cmap = plt.get_cmap('jet', 7)
real_roads_simp_joined2.plot(column="flow_class", categorical=True, cmap=cmap, legend=True, figsize=(10.0,8.0))

<AxesSubplot:>

In [42]:
# (5.11) Let's try do to a bit more of a professional job...
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
col_dict = { 1 : "gray", 2 : "blue", 3 : "green", 4 : "goldenrod", 5 : "orangered", 6 : "red" }
cmap2 = ListedColormap([col_dict[x] for x in col_dict.keys()])
real_roads_simp_joined2.plot(column="flow_class", categorical=True, cmap=cmap2, legend=True, figsize=(10.0,8.0))

<AxesSubplot:>